3.	Build statistical model(s) – also known as econometric model(s) – to explain how a set of explanatory variables each affect annual net savings. Comment on the results. Which explanatory variables have robust statistically significant effects in your model(s)? Do these results make sense?